In [9]:
using SparseArrays, LinearAlgebra, IndexedGraphs, DelimitedFiles
#using Pkg
#Pkg.activ§ate("../")
using Causality
using IntervalUnionArithmetic
using ProgressMeter

# Generation of Epidemics

In [5]:
T = 20.0 

20.0

In [6]:
folderstring = "../Scaling/try1/"
const Igen = GenerativeSI{GaussianRate,GaussianRate} 
const Igauss = GaussianInferentialSI

GaussianInferentialSI

In [ ]:
numsamplesList = [50,100,200,300,400,500,700,850,1000,1500,2000,3000,4000]
Nlist = [500,750,1000,1300,1500,2000]
λlist = [0.24,0.28,0.32,0.36,0.39,0.43]
@assert length(Nlist) == length(λlist)
for Ncount=1:length(Nlist)
    N = Nlist[Ncount]
    for trial = 1:2
        G = Causality.makeProximity(N,2.1/N)
        T = Float64(T)
        getpar(pseed,autoinf::GaussianRate,inf_in::GaussianRate) = 
            [fill(pseed, 1, N);
             fill(autoinf.a, 1, N); fill(autoinf.b, 1, N); fill(autoinf.c, 1, N);
             fill(inf_in.a,   1, N); fill(inf_in.b, 1, N); fill(inf_in.c, 1, N);
            ]

        getpargen(pseed, autoinf::GaussianRate, inf_out::GaussianRate) = 
            [pseed autoinf.a autoinf.b autoinf.c inf_out.a inf_out.b inf_out.c]

        #Initialize generation parameters
        ε = 1e-10
        λ = λlist[Ncount]
        pseed = 1/N
        autoinf = GaussianRate(ε, T/2, 1/ε)
        inf_in = GaussianRate(1.0, T/2, 1/ε)
        inf_out = GaussianRate(λ, T/2, 1/ε)

        θp = getpar(pseed, autoinf, inf_in);
        θpgen = getpargen(pseed, autoinf, inf_out);

        pseed_min = ε
        pseed_max = 1-ε
        rate_min = GaussianRate(ε  ,  -T,   ε)
        rate_max = GaussianRate(1/ε  ,  2T,   1/ε)

        θmin = getpar(pseed_min , rate_min, rate_min);
        θmax = getpar(pseed_max , rate_max, rate_max);

        rate_min = GaussianRate(ε  ,  -T,   ε)
        rate_max = GaussianRate(1/ε  ,  T,   T)

        θgenmin = getpargen(pseed_min, rate_min, rate_min);
        θgenmax = getpargen(pseed_max, rate_max, rate_max);


        V = fill(MaskedRate(UnitRate(),Causality.intervalUnion(0,T)), ne(G));
        Mp = StochasticModel(Igen, T, θp, G, θpgen, V);
        sample! = Sampler(Mp);
        xtrue = zeros(N)
        ni = 0
        while !(0.4<ni<0.6)
            sample!(xtrue)
            ni = sum(xtrue .< T)/N
        end
        ni
        nobs = Int(round(N/10))
        p = 1e-5 #p is mute
        O = [(ti = T; (i,xtrue[i] < ti,ti,p)) for i=1:nobs]
        for numsamples in numsamplesList
            T = Float64(T)
            ε = 2e-3
            autoinf = GaussianRate(ε, T/2, T);
            θp2gen = getpargen(pseed, autoinf, inf_out);
            inf_in = GaussianRate(1., T/2, 3*T );

            θp2 = getpar(pseed, autoinf, inf_in);
            Mp2 = StochasticModel(Igen, T, θp2, G, θp2gen, V);

            θ = getpar(pseed, autoinf, inf_in);
            autoinf = GaussianRate(0.01, T/2, T);
            M = StochasticModel(Igauss, T, θ, G, θp2gen, V);
            ProgressMeter.ijulia_behavior(:clear)
            F = descend!(Mp2, O; M=M, numsamples=numsamples, numiters=50, 
                     θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.1),
                     hyperdescender=SignDescender(0.));
            F = descend!(Mp2, O; M=M, numsamples=numsamples, numiters=50, 
                     θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.03),
                     hyperdescender=SignDescender(0.));
            statscau = prior(M, numsamples=10000);

            T = Float64(T)

            function marginal(i, t, stats)
                numsamp = size(stats,1)
                sum(stats[:,i] .< t)/numsamp
            end

            function reweighted_marginal(i, t, stats, weights)
                numsamp = size(stats,1)
                @assert numsamp == size(weights,1)
                weights ./= maximum(weights)
                sum(weights .* (stats[:,i] .< t))/sum(weights)
            end

            function tpr(xtrue, rank) 
                cumsum(xtrue[rank]) ./( cumsum(xtrue[rank])[end])
            end

            function fpr(xtrue, rank) 
                N = size(rank,1)
                return (range(1,N,length=N) .- cumsum(xtrue[rank])) ./ (range(1,N,length=N) .- cumsum(xtrue[rank]) )[end]
            end

            function ROC(xtrue, p)
                N = size(xtrue,1)
                rank = sortperm(p, rev=true)

                return fpr(xtrue, rank) , tpr(xtrue, rank)
            end

            function AUROC(ROC)
                N = size(ROC[1],1) 
                AU = 0
                for t = 1:N-1
                    AU += ROC[2][t] * (ROC[1][t+1] - ROC[1][t])
                end
                return AU
            end

            bins = Int(T)
            using Plots
            p_cau = zeros(N,bins)
            for i = 1:N
                for t = 1:bins
                   p_cau[i,t] = marginal(i, t*T/bins, statscau)
                end
            end

            T = Int(T)
            AU_curve=zeros(T)
            for t = 1:T
                cau_risk=zeros(N)
                for i=1:N
                   cau_risk[i] = marginal(i,t,statscau)
                end
                xt = xtrue .< t
                AU_curve[t] = AUROC(ROC(xt, cau_risk))
            end
            open("$(folderstring)try$(trial)numsamp$(numsamples)N$(N)cau.txt","w") do io
                    writedlm(io,AU_curve') 
            end
        end
    end
end

In [8]:
N = 2000
G = Causality.makeProximity(N,2.1/N)
    T = Float64(T)
    getpar(pseed,autoinf::GaussianRate,inf_in::GaussianRate) = 
        [fill(pseed, 1, N);
         fill(autoinf.a, 1, N); fill(autoinf.b, 1, N); fill(autoinf.c, 1, N);
         fill(inf_in.a,   1, N); fill(inf_in.b, 1, N); fill(inf_in.c, 1, N);
        ]

    getpargen(pseed, autoinf::GaussianRate, inf_out::GaussianRate) = 
        [pseed autoinf.a autoinf.b autoinf.c inf_out.a inf_out.b inf_out.c]

    #Initialize generation parameters
    ε = 1e-10
    λ = 0.43
    pseed = 1/N
    autoinf = GaussianRate(ε, T/2, 1/ε)
    inf_in = GaussianRate(1.0, T/2, 1/ε)
    inf_out = GaussianRate(λ, T/2, 1/ε)

    θp = getpar(pseed, autoinf, inf_in);
    θpgen = getpargen(pseed, autoinf, inf_out);

    pseed_min = ε
    pseed_max = 1-ε
    rate_min = GaussianRate(ε  ,  -T,   ε)
    rate_max = GaussianRate(1/ε  ,  2T,   1/ε)

    θmin = getpar(pseed_min , rate_min, rate_min);
    θmax = getpar(pseed_max , rate_max, rate_max);

    rate_min = GaussianRate(ε  ,  -T,   ε)
    rate_max = GaussianRate(1/ε  ,  T,   T)

    θgenmin = getpargen(pseed_min, rate_min, rate_min);
    θgenmax = getpargen(pseed_max, rate_max, rate_max);

    
    V = fill(MaskedRate(UnitRate(),Causality.intervalUnion(0,T)), ne(G));
    Mp = StochasticModel(Igen, T, θp, G, θpgen, V);
    sample! = Sampler(Mp);
    xtrue = zeros(N)
    ni = 0
    count=0
    for st = 1:100
        sample!(xtrue)
        ni = sum(xtrue .< T)/N
        count += ni
    end
    count / 100

0.5667949999999999